# Evaluate On Evaluation Set

## Summary

This notebook will do following steps:
 -  Pull validation data
 -  Load CTR predictions from CSV
 -  Load BaseBid Predictions for CSV
 -  Combine above using a formula
 -  Summarise Results

# Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint    
import seaborn as sns
from score_models import Scorer
import timeit
import random

%matplotlib inline

# Pull Validation Data

In [2]:
path = "C:/Users/Akis/OneDrive/Masters/Web Economics/Data/"
validation_df = pd.read_csv(path+'train.csv')

In [3]:
avgCTR =validation_df.loc[validation_df["click"] == 1].shape[0]/(validation_df.shape[0])
print(avgCTR)

0.0006646376573167722


# Load CTR Predictions

Need a function that will combine multiple dataframes into one

In [4]:
path='C:/Users/Akis/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/predictions/'
teamACTRData = pd.read_csv(path+'lr_pCTR.csv')
teamACTR=teamACTRData[['click_proba']]
teamACTR = teamACTR.rename(index=str, columns={"click_proba": "pCTR"})

# Load Basebid Predictions

Need a function that will combine multiple dataframes into one

In [5]:
teamABaseBidData = pd.read_csv(path+'ridge_basebid.csv')
teamABaseBid=teamABaseBidData[['base_bid']]
print(teamABaseBid.head(2))


    base_bid
0  80.350004
1  95.867239


# Build Bid Prices for all models

Need a function that will build a new dataframe conatining bids of all teams by combining the previous two dataframes.

In [6]:
#given formula
def Function1(resBaseBid,resCTR):
    avCTR = 0.5
    Bids=resBaseBid.values * (resCTR/avCTR)
    #print("Bids:",Bids)
    return Bids

In [7]:
#exponential CTR
def Function2(reBaseBid,reCTR):
    avCTR = 0.5
    reCTR=reCTR.copy(True)
    reCTR=(((reCTR+0.5)**2)-0.5)
    reCTR[reCTR < 0] = 0
    Bids=reBaseBid.values * (reCTR/avCTR)
    return Bids

In [8]:
#Zero bid for unsure values and high bidding on expected clicks
def Function3(rBaseBid,rCTR):
    avCTR = 0.5
    rCTR=rCTR.copy(True)
    rCTR[rCTR < 0.5] = 0
    #rCTR=(((rCTR+0.5)**4)-0.5)
    Bids=rBaseBid.values*1.2 * (rCTR/avCTR)
    return Bids

In [9]:
def Function4(BaseBid,CTR):
    Bids = BaseBid*0+301
    #Bids = 301
    return Bids

In [10]:
def Function5(BaseBid,CTR):
    Bids =  pd.DataFrame(np.random.randint(295, 301+1, BaseBid.shape[0]))
    #Bids = 301
    return Bids

In [11]:
resultsBids=pd.DataFrame(index=range(validation_df.shape[0]))
resultsBids['click']=validation_df['click']
resultsBids['payprice']=validation_df['payprice']

#resultsBids['TeamAFormula1'] = Function1(teamABaseBid,teamACTR).values

print(resultsBids.head(2))

   click  payprice
0      0        23
1      0        75


In [12]:
def EvaluateMultiple(results,BaseBidPred,CTRPred,function,budget):
    #print(function(BaseBidPred,CTRPred))
    results['TeamAFormula1'] = function(BaseBidPred,CTRPred).values
    s = Scorer()
    df_summary,df = s.set_df(results, ['payprice','TeamAFormula1'],budget,0)
    print(df_summary)
    return results

In [ ]:
#result = EvaluateMultiple(resultsBids,teamABaseBid,teamACTR,Function5,6250)

In [ ]:
start = timeit.timeit()

functions = [Function1, Function2, Function3, Function4, Function5]
for func in functions:
    print(func)
    result = EvaluateMultiple(resultsBids,teamABaseBid,teamACTR,func,6250000)

end = timeit.timeit()
print(end - start,"seconds")

<function Function1 at 0x000001E0818280D0>
